In [1]:
import pandas as pd
import math
import ipywidgets as widgets

from sidepanel import SidePanel

import regulus
from ipyregulus import GraphView, AxesCtrl, TreeView, ProjectionView, AttrFilter
from ipyregulus.core.axis import Axis
from ipyregulus.utils import *

### Data

In [2]:
name = 'gauss4'
data = regulus.load(name)

In [3]:
simplified = regulus.SimplifiedTree(data.tree, AttrFilter(attr='span'))

In [13]:
simplified.filter.value = -1

## Graph View

In [4]:
g = GraphView(tree=simplified)
g.color = 'z'
gp = SidePanel((g, 'graph'), title='GraphView')

#### Control axis size 

In [5]:
ctrl = AxesCtrl()
widgets.link((g, 'axes'), (ctrl, 'axes'))
ctrl

AxesCtrl(children=(UnboundedFloatSlider(value=200.0, description='x', max=200.0), UnboundedFloatSlider(value=2…

#### ... or add to the Graph Panel

In [6]:
gp.add(ctrl, header='Axes', expand=False)

### Tree View

In [7]:
tv = TreeView(simplified)
tp = SidePanel([(tv, 'tree'), (simplified.filter, 'Simplify')], title='TreeView')
widgets.link((tv.view, 'details'), (g,'show'));

#### _Select partitions_ 

### Let's select all partitions at a givin persistence

In [8]:
def at_persistence(v):
    g.show = [ p.id for p in data.tree.at_persistence(v)]

In [9]:
ps = widgets.FloatSlider(min=0, max=1, step=0.001, value=0.5)

In [10]:
widgets.interact(at_persistence, v=ps);

interactive(children=(FloatSlider(value=0.5, description='v', max=1.0, step=0.001), Output()), _dom_classes=('…

#### ... or add the slider to the Graph Panel

In [11]:
gp.add(ps, header='Persistence', expand=False)

## Project the points in the selected partitions

In [12]:
pr = ProjectionView(data.tree)
prp = SidePanel((pr, 'Projection'), title='ProjectionView', side='split-bottom', ref='TreeView')
widgets.link((g, 'show'), (pr, 'show'));

In [13]:
pr.color = 'z'

#### Coordiante the axes of the Graph and Projection views

In [14]:
widgets.link((g, 'axes'), (pr, 'axes'));

### set color based on the pt's partition 

In [14]:
def partition_color(pv):
    tree = pv.tree
    partitions = tree.find_partitions(pv.show)
    
    def in_partition(idx):
        for p in partitions:
            if idx in p.idx:
                return p.id
        return -1
    
    colors = list(map(in_partition, pv._index))
    return colors

In [15]:
pr.color = partition_color

In [35]:
pd.merge(left=data.pts.x, right=data.pts.values, left_index=True, right_index=True)

,x,y,z
0,0.938932,1.138623,0.614712
1,1.568838,1.597687,0.011168
2,1.217589,-0.109086,0.800305
3,0.636956,-1.727877,0.325892
4,-0.511846,0.853337,0.529962
5,0.006145,1.613801,0.292530
6,0.572005,-0.317035,0.511714
7,1.532263,0.462720,0.140347
8,0.872752,-0.905804,0.567934
9,-1.403087,0.432865,0.502102
